This board showcases what the default LLM monitoring template looks like. It logs synthetic data in the format used by the monitoring.openai integration.

In [ ]:
import uuid
from datetime import timedelta
import weave
weave.use_frontend_devmode()
from weave import ops_arrow
from weave.monitoring import monitor

In [ ]:
from weave.syndata_mon import random_predictions

In [ ]:
preds = random_predictions()

In [ ]:
# Convert synthetic data into the format used by the weave.monitoring.openai integration

spans = []
for pred in preds:
    span = monitor.Span('openai.api_resources.chat_completion.type.create',
                 inputs={
                     'messages':[
                         {"role": "user", "content": pred['prompt']}
                     ]
                 },
                 output={
                     'id': 'chatcmpl-%s' % uuid.uuid4(),
                     'object': 'chat.completion',
                     'created': pred['timestamp'].timestamp(),
                     'model': 'gpt-3.5-turbo-0613',
                     'choices': [
                         {
                             'index': 0,
                             'message': {
                                 'role': 'assistant',
                                 'content': pred['completion']
                             },
                             'finish_reason': 'stop'
                         }
                     ],
                     'usage': {
                         'prompt_tokens': pred['prompt_tokens'],
                         'completion_tokens': pred['completion_tokens'],
                         'total_tokens': pred['prompt_tokens'] + pred['completion_tokens']
                     }
                 },
                 attributes={
                     'api_key': 'sk-U4...yK7z',
                     'username': pred['username']
                 })
    span.start_time = pred['timestamp']
    span.end_time = pred['timestamp'] + timedelta(seconds=pred['latency'])
    spans.append({'timestamp': pred['timestamp'], **span.asdict()})

In [ ]:
# Save as a local table
oai_data = weave.save(ops_arrow.to_arrow(spans), 'oai_data')

In [ ]:
oai_data

In [ ]:
# Visualize the table
from weave import weave_internal
weave.use(oai_data['attributes'].keys().flatten().unique().map(lambda k: weave_internal.const('attribute.') + k))

In [ ]:
# Use the llm monitoring template to visualize the data
from weave.panels_py import panel_openai_monitor
board = panel_openai_monitor.board.raw_resolve_fn(oai_data)
board